In [ ]:
import numpy as np
from TMDP import TMDP
from algorithms import *
from model_functions import *
from algorithms import *
import matplotlib.pyplot as plt
import gymnasium as gym
from FrozenLake import *
import pandas as pd
#np.set_printoptions(precision=4)
import math
from utils import *

import torch
import mlflow
from CurriculumPMPO import CurriculumPMPO
import optuna

nrows = 30
nS = nrows**2
nA = 4
seed = get_current_seed()

gamma = .999999999
tau = 0.
num_bins = 10
#tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)
#mdp = TMDP(river, xi, tau=0., gamma=gamma, seed=seed)
env = FrozenLakeEnv(is_slippery=False, seed=seed, 
                    desc=generate_random_map(nrows, seed=seed), 
                    reward_shape=True,
                    num_bins=num_bins#, render_mode="human"
                    )
xi = np.ones(env.nS) * 1/env.nS
tmdp = TMDP(env, xi, tau=tau, gamma=gamma, seed=seed)

In [ ]:
folder_path = "./models/CurrPMPO/"
model_path = folder_path + "FrozenLake{}x{}_{}".format(nrows,nrows,num_bins)

In [ ]:
res = bellman_optimal_q(tmdp.env.P_mat, tmdp.env.reward, tmdp.gamma)
Q = res["Q"]
V = compute_V_from_Q(Q, get_policy(Q))
j_opt = compute_expected_j(V, tmdp.env.mu)
print("optimal performance: ",j_opt)

In [ ]:
episodes = 7900000
final_temp=1
lam = 1
batch_size = 32
param_decay=True
eps_ppo = 0.2
eps_model = compute_eps_model(gamma, tau, 790000)
epochs = 1
model_lr = 0.35869281717397965
pol_lr = 0.343385176875263757
temp = 5
tuning_rate = 0.99

In [ ]:
curr_PMPO = CurriculumPMPO(tmdp)
tmdp.update_tau(tau) # Reset the teleport rate        

In [ ]:
curr_PMPO.load_model(model_path)

In [ ]:
#V_Q = compute_V_from_Q(Q, get_policy(curr_PMPO.Q))
V_curr = compute_V_from_Q(Q, get_softmax_policy(curr_PMPO.theta, temperature=1e-4))

j_curr = compute_expected_j(V_curr, tmdp.env.mu)
#j_q_curr = compute_expected_j(V_Q, tmdp.env.mu)
j_opt = compute_expected_j(V, tmdp.env.mu)

print("optimal performance: ",j_opt)
print("curriculum performance: ", j_curr)
#print("Q curriculum performance: ",j_q_curr)


In [ ]:
import matplotlib.pyplot as plt
rewards = curr_PMPO.reward_records
# Generate recent 50 interval average
avg_rewards = []
for idx in range(len(rewards)):
    avg_list = np.empty(shape=(1,), dtype=int)
    if idx < 50:
        avg_list = rewards[:idx+1]
    else:
        avg_list = rewards[idx-49:idx+1]
    avg_rewards.append(np.average(avg_list))
# Plot
#plt.plot(rewards)
plt.plot(avg_rewards)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

exp_performances = curr_PMPO.exp_performances
# Generate recent 50 interval average
avg_performances = []
std_errors = []

for idx in range(len(exp_performances)):
    if idx < 500:
        avg_list = exp_performances[:idx+1]
    else:
        avg_list = exp_performances[idx-499:idx+1]

    average = np.mean(avg_list)
    std_dev = np.std(avg_list)
    n_samples = len(avg_list)
    sem = std_dev / np.sqrt(n_samples)  # Standard Error of the Mean
    std_errors.append(sem)

    avg_performances.append(average)

# Compute the upper and lower bounds for the 95% confidence interval
ci = 1.96  # Z-score for 95% confidence
upper_bound = [avg + ci * sem for avg, sem in zip(avg_performances, std_errors)]
lower_bound = [avg - ci * sem for avg, sem in zip(avg_performances, std_errors)]

# Plot
plt.figure(figsize=(10, 5))
#plt.plot(exp_performances, label='Exp Performances')
plt.plot(avg_performances, label='Moving Average', color='r')
plt.fill_between(range(len(avg_performances)), lower_bound, upper_bound, color='r', alpha=0.2, label='95% Confidence Interval')
plt.legend()
plt.title('Experiment Performances with Moving Average and Confidence Interval')
plt.xlabel('Index')
plt.ylabel('Performance')
plt.show()


In [ ]:
import numpy as np
rew = [[2,4,6,10], [3,5,7,9]]
max = np.max(rew, axis=0)
print(max)
avg = np.average(rew, axis=0)
print(avg)

In [ ]:
test_rewards = test_policies(tmdp, curr_PMPO.thetas, episodes=1000, temp=1e-4)
plt.plot(test_rewards)